# Import package

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil,subprocess
import os
import subprocess
import re
import mzbatch
from pyopenms import MSExperiment,MzMLFile,PeakPickerHiRes
import time
import pyopenms
from pyopenms import Plotting
import importlib as imp
imp.reload(mzbatch)

In [ ]:
# Set path
root_path = '../../Fulaoshi/230821_mzML/'
middle_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
batch_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
out_root_path = '../../Fulaoshi/230821_mzML_res/rerun240410'

# Construct the filtered secondary gallery

In [702]:
# con_MS2_info_all_df = pd.read_pickle(f'{out_root_path}/concat_MS2_info_best_df.pkl')
con_MS2_info_all_df = pd.read_pickle(f'{out_root_path}/concat_MS2_info_all_df.pkl')
con_MS2_info_all_df = con_MS2_info_all_df.reset_index(drop=True)
con_MS2_info_all_df = con_MS2_info_all_df.astype('str')
con_MS2_info_all_df = con_MS2_info_all_df[(con_MS2_info_all_df['filename']!="False")&(con_MS2_info_all_df['EV']!="204060")]
con_MS2_info_all_df.shape

(2430425, 19)

In [703]:
con_MS2_info_all_df = pd.read_pickle(f'{out_root_path}/concat_MS2_info_all_df.pkl')
con_MS2_info_all_df = con_MS2_info_all_df.astype('str')
con_MS2_info_all_df = con_MS2_info_all_df[(con_MS2_info_all_df['filename']!="False")]
con_MS2_info_all_df = con_MS2_info_all_df.reset_index(drop=True)
con_MS2_info_all_df['bscp']=con_MS2_info_all_df['batch']+'+'+con_MS2_info_all_df['sampleid'].astype(str)+'+'+con_MS2_info_all_df['CAS No.']+'+'+con_MS2_info_all_df['pos_neg']
con_MS2_info_all_df['filepath'] = out_root_path+'/'+con_MS2_info_all_df['batch']+'/NCE_compound/'+con_MS2_info_all_df['sampleid']+'/'+con_MS2_info_all_df['CAS No.']+"_NCE"+con_MS2_info_all_df['EV']+"_"+con_MS2_info_all_df['pos_neg']+'/'+con_MS2_info_all_df['filename']
print(con_MS2_info_all_df.shape)
con_MS2_info_all_df.head()

(2614930, 21)


batch sampleid   CAS No. EV pos_neg         mz  \
0  700_Amide      S98  576-42-1  1     neg  209.03029   
1  700_Amide      S98  576-42-1  1     neg  209.03029   
2  700_Amide      S98  576-42-1  1     neg  209.03029   
3  700_Amide      S98  576-42-1  1     neg  209.03029   
4  700_Amide      S98  576-42-1  1     neg  209.03029   

                  Product Name                                filename  \
0  D-Glucaric acid (potassium)  576-42-1_NCE1_neg_MS2_12037_802.39.csv   
1  D-Glucaric acid (potassium)   576-42-1_NCE1_neg_MS2_12081_805.6.csv   
2  D-Glucaric acid (potassium)   576-42-1_NCE1_neg_MS2_12126_808.7.csv   
3  D-Glucaric acid (potassium)  576-42-1_NCE1_neg_MS2_12171_811.87.csv   
4  D-Glucaric acid (potassium)   576-42-1_NCE1_neg_MS2_12221_815.4.csv   

  refer_valid ref_rt  ... is_max_ms1 ms1_intensity is_max_mol mol_intensity  \
0           1  826.2  ...          0      690369.0          0     488907.75   
1           1  826.2  ...          0  1036211.3125          0     752876.25   
2           1  826.2  ...          0   1241551.125          0      969256.9   
3           1  826.2  ...          0   1603654.625          0     1158187.0   
4           1  826.2  ...          0    1752998.75          0     1276483.2   

  is_max_base base_intensity       ms1_base_rate        ms1_mol_rate  \
0           0      488907.75   1.412063932306248   1.412063932306248   
1           0      752876.25   1.376336831584208   1.376336831584208   
2           0       969256.9  1.2809309121245358  1.2809309121245358   
3           0      1158187.0   1.384624956937006   1.384624956937006   
4           0      1276483.2  1.3733034245965792  1.3733034245965792   

                         bscp  \
0  700_Amide+S98+576-42-1+neg   
1  700_Amide+S98+576-42-1+neg   
2  700_Amide+S98+576-42-1+neg   
3  700_Amide+S98+576-42-1+neg   
4  700_Amide+S98+576-42-1+neg   

                                            filepath  
0  ../../Fulaoshi/230821_mzML_res/rerun240410/700...  
1  ../../Fulaoshi/230821_mzML_res/rerun240410/700...  
2  ../../Fulaoshi/230821_mzML_res/rerun240410/700...  
3  ../../Fulaoshi/230821_mzML_res/rerun240410/700...  
4  ../../Fulaoshi/230821_mzML_res/rerun240410/700...  

[5 rows x 21 columns]

In [705]:
con_MS2_info_all_df.columns

Index(['batch', 'sampleid', 'CAS No.', 'EV', 'pos_neg', 'mz', 'Product Name',
       'filename', 'refer_valid', 'ref_rt', 'ref_intensity', 'is_max_ms1',
       'ms1_intensity', 'is_max_mol', 'mol_intensity', 'is_max_base',
       'base_intensity', 'ms1_base_rate', 'ms1_mol_rate', 'bscp', 'filepath'],
      dtype='object')

In [550]:
low_diff = theo_mz - mzbatch.get_mz_lowlimit(theo_mz,diff=10)
mask_in_range = (((low_diff<one_ms2_df["mass_diff"]) & (one_ms2_df["mass_diff"]<= 0.99999)) | 
                ((3 <= one_ms2_df["mass_diff"]) & (one_ms2_df["mass_diff"] <= 13)) | 
                ((21 <= one_ms2_df["mass_diff"]) & (one_ms2_df["mass_diff"] <= 24)))
# 取反
mask_out_of_range = ~mask_in_range
one_ms2_df_filtered = one_ms2_df[mask_out_of_range]
one_ms2_df_filtered.shape

(42, 4)

In [546]:
theo_mz = float(theo_mz)
theo_mz - mzbatch.get_mz_range(theo_mz,diff=11)[0], theo_mz - mzbatch.get_mz_lowlimit(theo_mz,diff=11)

(0.006701007390006453, 0.006701007390006453)

In [ ]:
for i in con_MS2_info_all_df.index[0:]:
    print(con_MS2_info_all_df.loc[i,["batch","mz","bscp","filepath"]])
    if i%10000 == 0:
        print(i)
    batch,theo_mz,bscp,filepath = con_MS2_info_all_df.loc[i,["batch","mz","bscp","filepath"]]
    theo_mz = float(theo_mz)
    one_ms2_df = pd.read_csv(filepath,index_col=0)
    one_ms2_df = one_ms2_df[one_ms2_df["Intensity"]!=0]
    # Filter unreasonable ranges
    one_ms2_df["mass_diff"] = theo_mz-one_ms2_df["Mass"]
    low_diff = theo_mz - mzbatch.get_mz_lowlimit(theo_mz,diff=10)
    mask_in_range = (((low_diff<one_ms2_df["mass_diff"]) & (one_ms2_df["mass_diff"]<= 0.99999)) | 
                    ((3 <= one_ms2_df["mass_diff"]) & (one_ms2_df["mass_diff"] <= 13)) | 
                    ((21 <= one_ms2_df["mass_diff"]) & (one_ms2_df["mass_diff"] <= 24)))
    mask_out_of_range = ~mask_in_range # Take the inverse
    one_ms2_df = one_ms2_df[mask_out_of_range]
    
    newfile = filepath.replace("NCE_compound","NCE_compound_filter")
    end_index = newfile.rfind("/") + 1  # Find the position of the second last slash
    newpath = newfile[:end_index]

    if not os.path.exists(newpath):
        os.makedirs(newpath)
    while True:
        if os.path.exists(newpath):
            break

    try:
        filter_fragment_df = pd.read_csv(f'{out_root_path}/{batch}/NCE_filter_fragment/{bscp}.csv',index_col=0)
    except Exception as e:
        print(f"Error,{bscp}. Details: {e}")
        one_ms2_df.to_csv(newfile)
        continue
    filter_fragment_list = eval(filter_fragment_df.iloc[0,-1])

    lows,ups = mzbatch.get_mz_range(one_ms2_df["Mass"])
    lows,ups = np.array(lows), np.array(ups)

    temp_compare_df = pd.DataFrame({'cal_fragment':filter_fragment_list})

    valid_matrix = (temp_compare_df['cal_fragment'].values >= lows[:, None]) & \
                   (temp_compare_df['cal_fragment'].values <= ups[:, None])
    valid_list = valid_matrix.any(axis=1)
    one_ms2_df['valid_list'] = valid_list
    fragmentpeak_df = one_ms2_df[one_ms2_df["valid_list"]==True].loc[:,["Mass","Intensity"]]
    fragmentpeak_df = fragmentpeak_df.reset_index(drop=True)

    fragmentpeak_df.to_csv(newfile)


batch                                               873_Amide
mz                                                  144.06551
bscp                                 873_Amide+2+542-32-5+pos
filepath    ../../Fulaoshi/230821_mzML_res/rerun240410/873...
Name: 287808, dtype: object
报错,873_Amide+2+542-32-5+pos。内容：[Errno 2] No such file or directory: '../../Fulaoshi/230821_mzML_res/rerun240410/873_Amide/NCE_filter_fragment/873_Amide+2+542-32-5+pos.csv'


## Check whether the data is complete

In [640]:
for i in con_MS2_info_all_df.index[0:]:
    filter_filepath = con_MS2_info_all_df.loc[i,"filepath"].replace("NCE_compound","NCE_compound_filter")
    if not os.path.exists(filter_filepath):
        print("index:【", i,"】\n", con_MS2_info_all_df.loc[i,['batch', 'sampleid', 'CAS No.', 'EV', 'pos_neg', 'mz']])
        break 